In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta

In [2]:
#define the ticker symbol
tickerSymbol = 'MSFT'

In [3]:
#get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

In [4]:
#get the historical prices for this ticker
tickerDf = tickerData.history(interval='1d', start='2015-01-01', end='2024-01-01')

In [5]:
tickerDf

Open        High         Low       Close    Volume  \
Date                                                                   
2014-12-31   40.207044   40.817935   39.966129   39.966129  21552500   
2015-01-02   40.146805   40.800717   40.043557   40.232845  27913900   
2015-01-05   39.897287   40.207036   39.794039   39.862873  39673900   
2015-01-06   39.905894   40.224245   39.183148   39.277794  36447900   
2015-01-07   39.561726   39.974723   39.140126   39.776829  29114100   
...                ...         ...         ...         ...       ...   
2023-12-22  371.648813  373.140659  370.684084  372.543915  17091100   
2023-12-26  372.961653  374.891110  371.469806  372.623505  12673100   
2023-12-27  371.658773  373.021322  370.783552  372.036713  14905400   
2023-12-28  373.329619  374.413691  372.126205  373.240112  14327000   
2023-12-29  373.956211  375.109909  371.449920  373.996002  18723000   

            Dividends  Stock Splits  
Date                                 
2014-12-31        0.0             0  
2015-01-02        0.0             0  
2015-01-05        0.0             0  
2015-01-06        0.0             0  
2015-01-07        0.0             0  
...               ...           ...  
2023-12-22        0.0             0  
2023-12-26        0.0             0  
2023-12-27        0.0             0  
2023-12-28        0.0             0  
2023-12-29        0.0             0  

[2265 rows x 7 columns]

In [6]:
priceData = tickerDf.Open

In [7]:
plt.figure(figsize=(20,10))
plt.plot(priceData)
plt.title("%s Price Data"%tickerSymbol, fontsize=20)

Text(0.5, 1.0, 'MSFT Price Data')

In [8]:
priceData = priceData.asfreq(pd.infer_freq(priceData.index))

In [9]:
plt.figure(figsize=(20,10))
plt.plot(priceData)
for year in range(priceData.index[1].year, priceData.index[-1].year+1):
    plt.axvline(datetime(year,1,1), color='k', linestyle='--', alpha=0.2)
plt.title("%s Price Data"%tickerSymbol, fontsize=20)

Text(0.5, 1.0, 'MSFT Price Data')

## Basic Buying Protocol:
- Buy if stock increasing for __b__ consecutive days

## Basic Selling Protocols:
- Sell if stock decreasing for __s__ consecutive days __and__ we have made a profit

In [10]:
def get_buying_selling_days(price_data, b, s):
    
    #get the percent change day after day
    pct_change = price_data.pct_change()[1:]
    
    #this function checks the buying condition
    def buying_condition(sub_series):
        return (sub_series > 0).all()
    
    #this function checks the selling condition
    def selling_condition(sub_series):
        return (sub_series < 0).all()
    
    #get all buying days in the data
    buying_days = pct_change.rolling(b).apply(buying_condition)
    
    #get all potential selling days in the data
    potential_selling_days = pct_change.rolling(s).apply(selling_condition)
    
    #return a dictionary
    return {'buying_days': buying_days, 'potential_selling_days': potential_selling_days}

In [11]:
info_dict = get_buying_selling_days(priceData, 3, 3)

In [12]:
buying_days = info_dict['buying_days']
potential_selling_days = info_dict['potential_selling_days']

In [13]:
#create dataframe to store information
df_stocks = pd.DataFrame(index = buying_days.index)

In [14]:
#populate df with buying days, possible selling days, and price
df_stocks['buying_day'] = (buying_days == 1)
df_stocks['potential_selling_day'] = (potential_selling_days == 1)
df_stocks['price'] = priceData

#only keep days that are buying or possible selling days
df_stocks = df_stocks[(df_stocks.buying_day | df_stocks.potential_selling_day)]

In [15]:
def check_cumulative_percent_change(price_data, buy_date, potential_sell_date):
    """
    This helper function will check if the cumulative percent change
    between a buying and potential selling day yields overall growth
    """
    
    #get the percent change day after day
    pct_change = price_data.pct_change()[1:]
    
    sub_series = 1 + pct_change[buy_date + timedelta(hours=1): potential_sell_date]

    return sub_series.product() > 1

In [16]:
def get_investing_result(df_stocks, starting_funds, verbose=False):
    
    #get a copy of price data
    price_data = df_stocks.price
    
    #at start, not holding any shares
    holding = False
    
    #init vars
    current_funds = starting_funds
    current_shares = 0
    last_buy_date = None
    
    #init dict of buying and selling dates
    events_list = []
    
    #for each buying day and potential selling day...
    for date,data in df_stocks.iterrows():
        
        #if not currently holding shares, and this is a buying day...
        if (not holding) and data.buying_day:
            
            #calculate the number of shares we can buy
            num_shares_to_buy = int(current_funds / data.price)
            
            #update number of shares
            current_shares += num_shares_to_buy
            
            #decrease current funds
            current_funds -= num_shares_to_buy * data.price
            
            #set last buy date
            last_buy_date = date
            events_list.append(('b', date))
            
            #we are now holding shares
            holding = True
            
            if verbose:
                print('Bought %s shares at $%s on %s totaling $%s'%(num_shares_to_buy, data.price, date.date(), round(num_shares_to_buy*data.price,2)))
        
        #if you are holding shares, and this is a potential selling day...
        elif holding and data.potential_selling_day:
            
            #check to make sure we're making a profit
            if check_cumulative_percent_change(price_data, last_buy_date, date):
                #add to our current funds
                current_funds += current_shares * data.price
                
                if verbose:
                    print('Sold %s shares at $%s on %s totaling $%s'%(current_shares, data.price, date.date(), round(num_shares_to_buy*data.price,2)))
                    print('--------------------------------------')
                    
                #reset current shares
                current_shares = 0
                
                #we are no longer holding shares
                holding = False
                
                events_list.append(('s', date))
                
    #get the stock price at the end of the time span
    final_stock_price = price_data[-1]
        
    #get the final total value of all assets (funds + stock value)
    final_value = current_funds + final_stock_price * current_shares
    
    #return the percent change in value
    return round((final_value - starting_funds) / starting_funds,2), events_list

In [17]:
percent_change, events_list = get_investing_result(df_stocks, 10000, False)

In [18]:
print(round(percent_change*100),'%')

382 %


In [19]:
plt.figure(figsize=(20,10))
plt.plot(priceData)

y_lims = (int(priceData.min()*.95), int(priceData.max()*1.05))
shaded_y_lims = int(priceData.min()*.5), int(priceData.max()*1.5)

for idx, event in enumerate(events_list):
    color = 'red' if event[0] == 's' else 'blue'
    plt.axvline(event[1], color=color, linestyle='--', alpha=0.4)
    if event[0] == 's':
        plt.fill_betweenx(range(*shaded_y_lims), 
                          event[1], events_list[idx-1][1], color='k', alpha=0.1)

plt.title("%s Price Data"%tickerSymbol, fontsize=20)
plt.ylim(*y_lims)

(33.0, 400.0)